<a href="https://colab.research.google.com/github/HedersonSantos/Noticias/blob/main/exemplo_contrastive_com_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

class ContrastiveClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size, num_classes):
        super(ContrastiveClassifier, self).__init__()
        self.bert = bert_model
        self.fc1 = nn.Linear(hidden_size*4, hidden_size*2)
        self.fc2 = nn.Linear(hidden_size*2, num_classes)
    
    def forward(self, x1, x2, y=None):
        """
        x1, x2: input texts of shape (batch_size, max_seq_len)
        y: labels of shape (batch_size,)
        """
        # encode input texts with BERT
        _, h1 = self.bert(x1)
        _, h2 = self.bert(x2)
        
        # apply contrastive learning
        sim = F.cosine_similarity(h1, h2, dim=1)
        diff = torch.abs(h1 - h2)
        x = torch.cat([h1, h2, sim.unsqueeze(1), diff], dim=1)
        
        # feed through classification MLP
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        # calculate loss if labels are provided
        if y is not None:
            criterion = nn.CrossEntropyLoss()
            loss = criterion(x, y)
            return x, loss
        else:
            return x


Neste exemplo, ContrastiveClassifier é uma subclasse de nn.Module que define a arquitetura do modelo. Ele recebe um modelo BERT pré-treinado, hidden_size é o tamanho do vetor de características gerado pela camada oculta de BERT, e num_classes é o número de classes de saída (neste caso, 9).

O método forward define a lógica de passagem para a frente do modelo. Ele recebe duas entradas de texto x1 e x2 e as codifica com BERT. Em seguida, ele aplica a técnica de aprendizado por contraste, que compara as representações de texto de x1 e x2, e alimenta as diferenças e similaridades para um MLP de classificação.

Observe que o MLP de classificação tem duas camadas totalmente conectadas (fc1 e fc2) e usa a função de ativação ReLU. Além disso, a entrada para a camada fc1 é concatenada com a similaridade e diferença de x1 e x2. Isso é feito para incorporar a informação aprendida na etapa de aprendizado por contraste.

Se o parâmetro y for fornecido, o modelo calcula a perda de entropia cruzada (cross-entropy loss) entre as previsões x e as labels y e retorna tanto a saída x quanto a perda de treinamento. Caso contrário, ele retorna apenas a saída x.

Para treinar o modelo, você pode usar o algoritmo de otimização Adam e ajustar os hiperparâmetros, como a taxa de aprendizado e o número de épocas, conforme necessário:

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer

# initialize model and tokenizer
bert